# Simple Reinforcement Learning in Tensorflow Part 2: Policy Gradient Method
This tutorial contains a simple example of how to build a policy-gradient based agent that can solve the CartPole problem. For more information, see this [Medium post](https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-2-ded33892c724#.mtwpvfi8b).

For more Reinforcement Learning algorithms, including DQN and Model-based learning in Tensorflow, see my Github repo, [DeepRL-Agents](https://github.com/awjuliani/DeepRL-Agents). 

Parts of this tutorial are based on code by [Andrej Karpathy](https://gist.github.com/karpathy/a4166c7fe253700972fcbc77e4ea32c5) and [korymath](https://gym.openai.com/evaluations/eval_a0aVJrGSyW892vBM04HQA).

In [79]:
import numpy as np
import pickle as pickle
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import math
import os

### Loading the CartPole Environment
If you don't already have the OpenAI gym installed, use  `pip install gym` to grab it.

In [13]:
import gym
env = gym.make('CartPole-v1')

[2017-05-11 14:11:39,237] Making new env: CartPole-v1


What happens if we try running the environment with random actions? How well do we do? (Hint: not so well.)

In [14]:
env.reset()
random_episodes = 0
reward_sum = 0
while random_episodes < 10:
    env.render()
    observation, reward, done, _ = env.step(np.random.randint(0,2))
    reward_sum += reward
    if done:
        random_episodes += 1
        print ("Reward for this episode was:",reward_sum)
        reward_sum = 0
        env.reset()
env.render(close=True)

Reward for this episode was: 11.0
Reward for this episode was: 21.0
Reward for this episode was: 13.0
Reward for this episode was: 16.0
Reward for this episode was: 19.0
Reward for this episode was: 16.0
Reward for this episode was: 16.0
Reward for this episode was: 14.0
Reward for this episode was: 43.0
Reward for this episode was: 14.0


The goal of the task is to achieve a reward of 200 per episode. For every step the agent keeps the pole in the air, the agent recieves a +1 reward. By randomly choosing actions, our reward for each episode is only a couple dozen. Let's make that better with RL!

### Setting up our Neural Network agent
This time we will be using a Policy neural network that takes observations, passes them through a single hidden layer, and then produces a probability of choosing a left/right movement. To learn more about this network, see [Andrej Karpathy's blog on Policy Gradient networks](http://karpathy.github.io/2016/05/31/rl/).

In [15]:
# hyperparameters
H = 10 # number of hidden layer neurons
batch_size = 5 # every how many episodes to do a param update?
learning_rate = 1e-2 # feel free to play with this to train faster or more stably.
gamma = 0.99 # discount factor for reward
nb_layers=2; #BE SURE TO UPDATE THIS VARIABLE FOR CORRECT SAVE 

D = 4 # input dimensionality

In [117]:
A=[5,3,4]
def array_to(arr):
    s=str(arr)
    s=s.replace('[',' ')
    s=s.replace(']',' ')
    s=s.replace(' ','')
    s=s.replace(',','_')
    return s
array_to(A)
B={}
B['test']=5
print(B)

{'test': 5}


In [151]:
class Net(object):
    def __init__(self, size_layers,layer_function,batch_size=5,learning_rate = 1e-2,gamma = 0.99  ):
        self.size_layers=size_layers
        self.layer_function=layer_function
        self.batch_size=batch_size
        self.learning_rate=learning_rate
        self.gamma=gamma
        
        self.Ws=[]
        tf.reset_default_graph()
        #This defines the network as it goes from taking an observation of the environment to 
        #giving a probability of chosing to the action of moving left or right.
        
        size_layers.append(1)
        self.observations = tf.placeholder(tf.float32, [None,size_layers[0]] , name="input_x")

        Wprev = tf.get_variable("W1", shape=[size_layers[0], size_layers[1]],
                   initializer=tf.contrib.layers.xavier_initializer())
        self.Ws.append(Wprev)
        layerprev = layer_function[0](tf.matmul(self.observations,Wprev))

        for i in range(1,len(size_layers)-1):
            var_name= 'W'+str(i+1)
            Wi =  tf.get_variable(var_name, shape=[size_layers[i], size_layers[i+1]],
                   initializer=tf.contrib.layers.xavier_initializer())
            self.Ws.append(Wi)
            layeri = layer_function[i](tf.matmul(layerprev,Wi))

            Wprev=Wi
            layerprev=layeri

        self.probability = layeri

        #From here we define the parts of the network needed for learning a good policy.
        self.tvars = tf.trainable_variables()
        self.input_y = tf.placeholder(tf.float32,[None,1], name="input_y")
        self.advantages = tf.placeholder(tf.float32,name="reward_signal")

        # The loss function. This sends the weights in the direction of making actions 
        # that gave good advantage (reward over time) more likely, and actions that didn't less likely.
        loglik = tf.log(self.input_y*(self.input_y - self.probability) + (1 - self.input_y)*(self.input_y + self.probability))
        loss = -tf.reduce_mean(loglik * self.advantages) 
        self.newGrads = tf.gradients(loss,self.tvars)

        # Once we have collected a series of gradients from multiple episodes, we apply them.
        # We don't just apply gradients after every episode in order to account for noise in the reward signal.
        adam = tf.train.AdamOptimizer(learning_rate=learning_rate) # Our optimizer

        self.batchGrad=[]
        for j in range(1,len(self.size_layers)):
            varname='batch_grad'+str(j)
            self.batchGrad.append(tf.placeholder(tf.float32,name=varname))# Placeholders to send the final gradients through when we update.
        self.updateGrads = adam.apply_gradients(zip(self.batchGrad,self.tvars))
        
        self.init = tf.global_variables_initializer()
        self.saver = tf.train.Saver()
        self.sess = tf.Session()
    def array_to_str(self,arr):
        s=str(arr)
        s=s.replace('[',' ')
        s=s.replace(']',' ')
        s=s.replace(' ','')
        s=s.replace(',','_')
        return s
    #stop_reward: stop criteria based on average reward 
    def train_network(self,total_episodes,stop_reward,loadModel,saveModel):
        xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[]
        running_reward = None
        reward_sum = 0
        episode_number = 1
        
        folder_name='models/'+'H_'+self.array_to_str(self.size_layers)+'/'
        model_name = folder_name+'model'
        name = model_name+'.meta'
        fullrendering = False
        rendering=False

        # Launch the graph
        with self.sess as sess:
            sess.run(self.init)
            if loadModel:
                self.saver.restore(sess, model_name)
            observation = env.reset() # Obtain an initial observation of the environment

            # Reset the gradient placeholder. We will collect gradients in 
            # gradBuffer until we are ready to update our policy network. 
            gradBuffer = sess.run(self.tvars)
            for ix,grad in enumerate(gradBuffer):
                gradBuffer[ix] = grad * 0
            while episode_number <= total_episodes:

                # Rendering the environment slows things down, 
                # so let's only look at it once our agent is doing a good job.
                if (reward_sum/batch_size > 100 or fullrendering == True) and rendering : 
                    env.render()
                    rendering = True

                # Make sure the observation is in a shape the network can handle.
                x = np.reshape(observation,[1,D])

                # Run the policy network and get an action to take. 
                tfprob = sess.run(self.probability,feed_dict={self.observations: x})
                action = 1 if np.random.uniform() < tfprob else 0

                xs.append(x) # observation
                y = 1 if action == 0 else 0 # a "fake label"
                ys.append(y)

                # step the environment and get new measurements
                observation, reward, done, info = env.step(action)
                reward_sum += reward

                drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

                if done: 
                    episode_number += 1
                    # stack together all inputs, hidden states, action gradients, and rewards for this episode
                    epx = np.vstack(xs)
                    epy = np.vstack(ys)
                    epr = np.vstack(drs)
                    tfp = tfps
                    xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[] # reset array memory

                    # compute the discounted reward backwards through time
                    discounted_epr = discount_rewards(epr)
                    # size the rewards to be unit normal (helps control the gradient estimator variance)
                    discounted_epr -= np.mean(discounted_epr)
                    discounted_epr /= np.std(discounted_epr)

                    # Get the gradient for this episode, and save it in the gradBuffer
                    tGrad = sess.run(self.newGrads,feed_dict={self.observations: epx, self.input_y: epy, self.advantages: discounted_epr})
                    for ix,grad in enumerate(tGrad):
                        gradBuffer[ix] += grad

                    # If we have completed enough episodes, then update the policy network with our gradients.
                    if episode_number % batch_size == 0: 
                        feed_dict={}
                        for my_index in range(0,len(self.size_layers)-1):
                            feed_dict[self.batchGrad[my_index]]=gradBuffer[my_index]
                        sess.run(self.updateGrads,feed_dict)
                        for ix,grad in enumerate(gradBuffer):
                            gradBuffer[ix] = grad * 0

                        # Give a summary of how well our network is doing for each batch of episodes.
                        running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                        print('Average reward for episode',episode_number,': %f.  Total average reward %f.' % (reward_sum/batch_size, running_reward/batch_size))

                        if reward_sum/batch_size > stop_reward: 
                            print ("Task solved in",episode_number,'episodes!')
                            break

                        reward_sum = 0

                    observation = env.reset()
            #save model
            if saveModel:
                if not os.path.exists(folder_name):
                    os.makedirs(folder_name)
                self.saver.save(sess, model_name)
                with tf.variable_scope("",reuse=True):
                    print(tf.get_variable("W1").eval())

        print (episode_number,'Episodes completed.')

#test_network = Net([4,10],[tf.nn.relu,tf.nn.sigmoid])
test_network = Net([4,10,10],[tf.nn.relu,tf.nn.relu,tf.nn.sigmoid])

#test_network.train_network(number_episods,max_score,Load,Save)
test_network.train_network(1000,200,True,True)

INFO:tensorflow:Restoring parameters from models/H_4_10_10_1/model


[2017-05-11 18:29:02,315] Restoring parameters from models/H_4_10_10_1/model


Average reward for episode 5 : 214.000000.  Total average reward 214.000000.
Task solved in 5 episodes!
[[ 0.54381543  0.03872322  0.51595801  0.21302424 -0.11190394 -0.53458869
   0.65915322 -0.24182701  0.62814724 -0.14884129]
 [ 0.49174616  0.23888563 -0.44936427  0.2078806   0.37950397 -0.07419132
   0.58289623  0.47441447  0.05068268  0.02965043]
 [-0.94739473  0.58078051  0.20058528  0.13413151  0.11316664  0.66154718
   0.28853607  0.9173215  -0.2145074   0.00837148]
 [-0.39093953  0.52243435  0.32740548 -0.68446898  0.31282112  0.93070328
   0.30169961  0.42239973 -1.14487195  0.06236639]]
5 Episodes completed.


In [112]:
# size_layers: array containing the size of the layers [], first value should be 4 (size of input !!)
# layer_function: function apply the the output of each layer (the size of this array should be size (size_hlayers) + 1)
# batch_size: every how many episodes to do a param update?
# learning_rate: feel free to play with this to train faster or more stably.
# gamma: discount factor for reward
# D:input dimensionality

#example= init_network([4,10],[tf.nn.relu,tf.nn.sigmoid] )
def init_network(size_layers,layer_function,batch_size=5,learning_rate = 1e-2,gamma = 0.99  ):
    tf.reset_default_graph()
    #This defines the network as it goes from taking an observation of the environment to 
    #giving a probability of chosing to the action of moving left or right.
    
    size_layers.append(1)
    observations = tf.placeholder(tf.float32, [None,size_layers[0]] , name="input_x")
    
    Wprev = tf.get_variable("W1", shape=[size_layers[0], size_layers[1]],
               initializer=tf.contrib.layers.xavier_initializer())
    layerprev = layer_function[0](tf.matmul(observations,Wprev))
    
    for i in range(1,len(size_layers)-1):
        var_name= 'W'+str(i+1)
        Wi =  tf.get_variable(var_name, shape=[size_layers[i], size_layers[i+1]],
               initializer=tf.contrib.layers.xavier_initializer())
        layeri = layer_function[i](tf.matmul(layerprev,Wi))
        
        Wprev=Wi
        layerprev=layeri
        
    probability = layeri

    #From here we define the parts of the network needed for learning a good policy.
    tvars = tf.trainable_variables()
    input_y = tf.placeholder(tf.float32,[None,1], name="input_y")
    advantages = tf.placeholder(tf.float32,name="reward_signal")

    # The loss function. This sends the weights in the direction of making actions 
    # that gave good advantage (reward over time) more likely, and actions that didn't less likely.
    loglik = tf.log(input_y*(input_y - probability) + (1 - input_y)*(input_y + probability))
    loss = -tf.reduce_mean(loglik * advantages) 
    newGrads = tf.gradients(loss,tvars)

    # Once we have collected a series of gradients from multiple episodes, we apply them.
    # We don't just apply gradients after every episode in order to account for noise in the reward signal.
    adam = tf.train.AdamOptimizer(learning_rate=learning_rate) # Our optimizer
    
    batchGrad=[]
    for j in range(1,len(size_layers)):
        varname='batch_grad'+str(j)
        batchGrad.append(tf.placeholder(tf.float32,name=varname))# Placeholders to send the final gradients through when we update.
    updateGrads = adam.apply_gradients(zip(batchGrad,tvars))

init_network([4,10],[tf.nn.relu,tf.nn.sigmoid] )

### Advantage function
This function allows us to weigh the rewards our agent recieves. In the context of the Cart-Pole task, we want actions that kept the pole in the air a long time to have a large reward, and actions that contributed to the pole falling to have a decreased or negative reward. We do this by weighing the rewards from the end of the episode, with actions at the end being seen as negative, since they likely contributed to the pole falling, and the episode ending. Likewise, early actions are seen as more positive, since they weren't responsible for the pole falling.

In [39]:
def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    #for t in reversed(xrange(0, r.size)):
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

### Running the Agent and Environment

Here we run the neural network agent, and have it act in the CartPole environment.

In [113]:
xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 1
total_episodes = 1000
#init = tf.initialize_all_variables()
init = tf.global_variables_initializer()

folder_name='models/H_4_10_1/'
model_name = folder_name+'model'
name = model_name+'.meta'
loadModel=True;
saveModel=False;
fullrendering = False
rendering=False

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    if loadModel:
        new_saver = tf.train.import_meta_graph(name,clear_devices=True)
        #new_saver.restore(sess, model_name)
        saver.restore(sess,tf.train.latest_checkpoint(folder_name))
    observation = env.reset() # Obtain an initial observation of the environment

    # Reset the gradient placeholder. We will collect gradients in 
    # gradBuffer until we are ready to update our policy network. 
    gradBuffer = sess.run(tvars)
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
    
    while episode_number <= total_episodes:
        
        # Rendering the environment slows things down, 
        # so let's only look at it once our agent is doing a good job.
        if (reward_sum/batch_size > 100 or fullrendering == True) and rendering : 
            env.render()
            rendering = True
            
        # Make sure the observation is in a shape the network can handle.
        x = np.reshape(observation,[1,D])
        
        # Run the policy network and get an action to take. 
        tfprob = sess.run(probability,feed_dict={observations: x})
        action = 1 if np.random.uniform() < tfprob else 0
        
        xs.append(x) # observation
        y = 1 if action == 0 else 0 # a "fake label"
        ys.append(y)

        # step the environment and get new measurements
        observation, reward, done, info = env.step(action)
        reward_sum += reward

        drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

        if done: 
            episode_number += 1
            # stack together all inputs, hidden states, action gradients, and rewards for this episode
            epx = np.vstack(xs)
            epy = np.vstack(ys)
            epr = np.vstack(drs)
            tfp = tfps
            xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[] # reset array memory

            # compute the discounted reward backwards through time
            discounted_epr = discount_rewards(epr)
            # size the rewards to be unit normal (helps control the gradient estimator variance)
            discounted_epr -= np.mean(discounted_epr)
            discounted_epr /= np.std(discounted_epr)
            
            # Get the gradient for this episode, and save it in the gradBuffer
            tGrad = sess.run(newGrads,feed_dict={observations: epx, input_y: epy, advantages: discounted_epr})
            for ix,grad in enumerate(tGrad):
                gradBuffer[ix] += grad
                
            # If we have completed enough episodes, then update the policy network with our gradients.
            if episode_number % batch_size == 0: 
                sess.run(updateGrads,feed_dict={W1Grad: gradBuffer[0],W2Grad:gradBuffer[1]})
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0
                
                # Give a summary of how well our network is doing for each batch of episodes.
                running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                print('Average reward for episode',episode_number,': %f.  Total average reward %f.' % (reward_sum/batch_size, running_reward/batch_size))
                
                if reward_sum/batch_size > 200: 
                    print ("Task solved in",episode_number,'episodes!')
                    break
                    
                reward_sum = 0
            
            observation = env.reset()
    #save model
    if saveModel:
        saver = tf.train.Saver([W1,W2])
        saver.save(sess, model_name)

print (episode_number,'Episodes completed.')


INFO:tensorflow:Restoring parameters from models/H_4_10_1/model


[2017-05-11 17:37:49,102] Restoring parameters from models/H_4_10_1/model


ValueError: Fetch argument <tf.Variable 'W1:0' shape=(4, 10) dtype=float32_ref> cannot be interpreted as a Tensor. (Tensor Tensor("W1:0", shape=(4, 10), dtype=float32_ref) is not an element of this graph.)

As you can see, the network not only does much better than random actions, but achieves the goal of 200 points per episode, thus solving the task!